In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 3_anisotropic_df_likelihoods.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Compute likelihoods for the anisotropic DFs.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os, dill as pickle
import pdb, copy, glob, time

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu
from astropy import constants as apc

## Analysis
import scipy.stats
import scipy.interpolate

## galpy
from galpy import orbit
from galpy import potential
from galpy import actionAngle as aA
from galpy import df
from galpy import util as gputil

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import cutout as pcutout
from tng_dfs import densprofile as pdens
from tng_dfs import fitting as pfit
from tng_dfs import io as pio
from tng_dfs import kinematics as pkin
from tng_dfs import util as putil
from tng_dfs import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/5_compare_distribution_functions/3_anisotropic_df_likelihoods/')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Compute DF values for each DF

In [ ]:
verbose = True

# Pathing
dens_fitting_dir = os.path.join(fitting_dir_base,'density_profile')
df_fitting_dir = os.path.join(fitting_dir_base,'distribution_function')
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Get the orbits
sample_data_cb = np.load(os.path.join(analysis_dir,'sample_data_cb.npy'),
    allow_pickle=True)
sample_data_om = np.load(os.path.join(analysis_dir,'sample_data_om.npy'),
    allow_pickle=True)
sample_data_om2 = np.load(os.path.join(analysis_dir,'sample_data_om2.npy'),
    allow_pickle=True)

# Potential interpolator version
interpot_version = 'all_star_dm_enclosed_mass'

# Stellar halo density information
stellar_halo_density_version = 'poisson_twopower_softening'
stellar_halo_density_ncut = 500

# Stellar halo rotation information
stellar_halo_rotation_dftype = 'tanh_rotation'
stellar_halo_rotation_version = 'asymmetry_fit'
stellar_halo_rotation_ncut = 500

# Define density profiles
stellar_halo_densfunc = pdens.TwoPowerSpherical()

df_val_data_cb = []
df_val_data_om = []
df_val_data_om2 = []

for i in range(n_mw):
    # if i > 0: continue

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    pid = co.get_property('stars','ParticleIDs')

    # Load the interpolator for the sphericalized potential
    interpolator_filename = os.path.join(dens_fitting_dir,
        'spherical_interpolated_potential/',interpot_version,
        str(z0_sid),'interp_potential.pkl')
    with open(interpolator_filename,'rb') as handle:
        interpot = pickle.load(handle)

    for j in range(n_major):
        # if j > 0: continue
        if verbose: print(f'Calculating anisotropy profiles for MW analog '
                          f'{i+1}/{n_mw}, merger {j+1}/{n_major}', end='\r')

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        upid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
        indx = np.where(np.isin(pid,upid))[0]

        # Get energy and angular momentum
        orbs = co.get_orbs('stars')[indx]
        n_star = len(orbs)
        rs = orbs.r().to_value(apu.kpc)
        masses = co.get_masses('stars')[indx].to_value(apu.Msun)
        L = np.sum(orbs.L().to_value(apu.kpc*apu.km/apu.s)**2,axis=1)**0.5
        Lz = orbs.Lz().to_value(apu.kpc*apu.km/apu.s)
        pe = co.get_potential_energy('stars')[indx].to_value(apu.km**2/apu.s**2)
        vels = co.get_velocities('stars')[indx].to_value(apu.km/apu.s)
        vmag = np.linalg.norm(vels,axis=1)
        energy = pe + 0.5*vmag**2

        # Mask the input orbits to only include those with radius greater
        # than the softening length
        r_softening = putil.get_softening_length('stars', z=0, physical=True)
        mask = rs > r_softening
        orbs = orbs[mask]
        rs = rs[mask]
        masses = masses[mask]
        L = L[mask]
        Lz = Lz[mask]
        pe = pe[mask]
        vels = vels[mask]
        vmag = vmag[mask]
        energy = energy[mask]
        n_star = len(orbs)

        # Get the stellar halo density profile (denspot for the DF)
        stellar_halo_density_filename = os.path.join(dens_fitting_dir,
            'stellar_halo/',stellar_halo_density_version,str(z0_sid),
            'merger_'+str(j+1)+'/', 'sampler.pkl')
        denspot = pfit.construct_pot_from_fit(
            stellar_halo_density_filename, stellar_halo_densfunc, 
            stellar_halo_density_ncut, ro=ro, vo=vo)
        
        # Get the stellar halo rotation kernel
        stellar_halo_rotation_filename = os.path.join(df_fitting_dir,
            stellar_halo_rotation_dftype,stellar_halo_rotation_version,
            str(z0_sid),'merger_'+str(j+1)+'/', 'sampler.pkl')
        stellar_halo_k, stellar_halo_chi = \
            pio.median_params_from_emcee_sampler(stellar_halo_rotation_filename,
                ncut=stellar_halo_rotation_ncut)
        
        # Scale the input energies by the potential energy of interpot at the 
        # stellar half-mass radius
        rhalf = pkin.half_mass_radius(rs, masses)
        rhalf_perc = 0.05
        rhalf_mask = np.abs(rs-rhalf) < rhalf_perc*rhalf
        rhalf_pe_star = np.median(pe[rhalf_mask])
        rhalf_pe_interpot = potential.evaluatePotentials(
            interpot, rhalf*apu.kpc, 0.).to_value(apu.km**2/apu.s**2)
        rhalf_pe_offset = rhalf_pe_star - rhalf_pe_interpot

        ########### Compute the DF values for the constant beta DFs ###########

        # Load the distribution function and wrangle
        df_type = 'constant_beta'
        df_version = 'df_density_softening'
        df_filename = os.path.join(df_fitting_dir,df_type,df_version,
            str(z0_sid),'merger_'+str(j+1),'df.pkl')
        with open(df_filename,'rb') as handle:
            dfcb = pickle.load(handle)
        dfcb = pkin.reconstruct_anisotropic_df(dfcb, interpot, denspot)

        # Compute the internal energy and angular momentum
        energy_offset = energy - rhalf_pe_offset
        energy_offset_internal = energy_offset / (vo**2)
        L_internal = L/ro/vo
        
        # Compute the value of the DF speedily using the interpolator
        fE = dfcb._fE_interp(energy_offset_internal)
        fL = L_internal**(-2*dfcb._beta)
        fLz = (stellar_halo_k*np.tanh(Lz/stellar_halo_chi)-stellar_halo_k+1)
        f = fE*fL*fLz

        # Do the self-similar assessment of the DF
        # Create the samples for the DF
        mask = (sample_data_cb['z0_sid'] == z0_sid) &\
               (sample_data_cb['major_acc_sid'] == major_acc_sid) &\
               (sample_data_cb['major_mlpid'] == major_mlpid) &\
               (sample_data_cb['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]
        sample = sample_data_cb['sample'][indx]

        # Compute the likelihoods for the self-similar DF
        pe_offset_sample = 0. # PE offset for DF samples should be 0
        energy_offset_sample = sample.E(pot=interpot).to_value(apu.km**2/apu.s**2) - pe_offset_sample
        energy_offset_internal_sample = energy_offset_sample / (vo**2)
        L_sample = (np.sum(sample.L()**2,1)**0.5).to_value(apu.kpc*apu.km/apu.s)
        L_internal_sample = L_sample/ro/vo
        Lz_sample = sample.Lz().to_value(apu.kpc*apu.km/apu.s)
        fE_sample = dfcb._fE_interp(energy_offset_internal_sample)
        fL_sample = L_internal_sample**(-2*dfcb._beta)
        fLz_sample = (stellar_halo_k*np.tanh(Lz_sample/stellar_halo_chi)-stellar_halo_k+1)
        f_sample = fE_sample*fL_sample*fLz_sample

        # Save the values
        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            fE,
            fL,
            fLz,
            f,
            fE_sample,
            fL_sample,
            fLz_sample,
            f_sample
        )
        df_val_data_cb.append(_data)

        ########## Compute the DF values for the Osipkov-Merritt DFs ##########

        # Load the distribution function and wrangle
        df_type = 'osipkov_merritt'
        df_version = 'df_density_softening'
        df_filename = os.path.join(df_fitting_dir,df_type,df_version,
            str(z0_sid),'merger_'+str(j+1),'df.pkl')
        with open(df_filename,'rb') as handle:
            dfom = pickle.load(handle)
        dfom = pkin.reconstruct_anisotropic_df(dfom, interpot, denspot)

        # Compute the energy and Q
        energy_offset = energy - rhalf_pe_offset
        Q_offset = -energy_offset - 0.5*L**2/(dfom._ra*ro)**2
        Q_offset_internal = Q_offset / (vo**2)
        
        # Compute the value of the DF speedily using the interpolator
        fQ = np.exp( dfom._logfQ_interp(Q_offset_internal) )
        fLz = (stellar_halo_k*np.tanh(Lz/stellar_halo_chi)-stellar_halo_k+1)
        f = fQ*fLz

        # Do the self-similar assessment of the DF
        # Create the samples for the DF
        mask = (sample_data_om['z0_sid'] == z0_sid) &\
               (sample_data_om['major_acc_sid'] == major_acc_sid) &\
               (sample_data_om['major_mlpid'] == major_mlpid) &\
               (sample_data_om['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]
        sample = sample_data_om['sample'][indx]

        # Compute the likelihoods for the self-similar DF
        pe_offset_sample = 0. # PE offset for DF samples should be 0
        energy_offset_sample = sample.E(pot=interpot).to_value(apu.km**2/apu.s**2) - pe_offset_sample
        L_sample = (np.sum(sample.L()**2,1)**0.5).to_value(apu.kpc*apu.km/apu.s)
        Q_offset_sample = -energy_offset_sample - 0.5*L_sample**2/(dfom._ra*ro)**2
        Q_offset_internal_sample = Q_offset_sample / (vo**2)
        Lz_sample = sample.Lz().to_value(apu.kpc*apu.km/apu.s)
        fQ_sample = np.exp( dfom._logfQ_interp(Q_offset_internal_sample) )
        fLz_sample = (stellar_halo_k*np.tanh(Lz_sample/stellar_halo_chi)-stellar_halo_k+1)
        f_sample = fQ_sample*fLz_sample

        # Save the values
        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            fE,
            fL,
            fLz,
            f,
            fE_sample,
            fL_sample,
            fLz_sample,
            f_sample
        )
        df_val_data_om.append(_data)

        ########## Compute the DF values for the OM2 DFs ##########

        # Load the distribution function and wrangle
        df_type = 'osipkov_merritt_2_combination'
        df_version = 'ra_N10_01_to_300_softening'
        df_filename = os.path.join(df_fitting_dir,df_type,df_version,
            str(z0_sid),'merger_'+str(j+1),'df.pkl')
        with open(df_filename,'rb') as handle:
            data = pickle.load(handle)
        dfoms = data[0]
        ras = data[1]
        kom = data[2]
        for k in range(len(dfoms)):
            dfoms[k] = pkin.reconstruct_anisotropic_df(dfoms[k], interpot, 
                denspot)
        
        # Compute the energy and Q
        energy_offset = energy - rhalf_pe_offset
        Q_offset = -energy_offset - 0.5*L**2/(dfom._ra*ro)**2
        Q_offset_internal = Q_offset / (vo**2)

        # Compute the value of the DF speedily using the interpolator
        fQ1 = np.exp( dfoms[0]._logfQ_interp(Q_offset_internal) )
        fQ2 = np.exp( dfoms[1]._logfQ_interp(Q_offset_internal) )
        fLz = (kom*np.tanh(Lz/stellar_halo_chi)-kom+1)
        f = (fQ1*kom + fQ2*(1-kom))*fLz

        # Do the self-similar assessment of the DF
        # Create the samples for the DF
        mask = (sample_data_om2['z0_sid'] == z0_sid) &\
                (sample_data_om2['major_acc_sid'] == major_acc_sid) &\
                (sample_data_om2['major_mlpid'] == major_mlpid) &\
                (sample_data_om2['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]
        sample = sample_data_om2['sample'][indx]

        # Compute the likelihoods for the self-similar DF
        pe_offset_sample = 0. # PE offset for DF samples should be 0
        energy_offset_sample = sample.E(pot=interpot).to_value(apu.km**2/apu.s**2) - pe_offset_sample
        L_sample = (np.sum(sample.L()**2,1)**0.5).to_value(apu.kpc*apu.km/apu.s)
        ra_sample = np.zeros(len(sample))
        ra_sample[:int(n_star*kom)] = dfoms[0]._ra
        ra_sample[int(n_star*kom):] = dfoms[1]._ra
        Q_offset_sample = -energy_offset_sample - 0.5*L_sample**2/(ra_sample*ro)**2
        Q_offset_internal_sample = Q_offset_sample / (vo**2)
        Lz_sample = sample.Lz().to_value(apu.kpc*apu.km/apu.s)
        fQ_sample = np.zeros(len(sample))
        fQ_sample[:int(n_star*kom)] = np.exp( 
            dfoms[0]._logfQ_interp(Q_offset_internal_sample[:int(n_star*kom)]) )
        fQ_sample[int(n_star*kom):] = np.exp(
            dfoms[1]._logfQ_interp(Q_offset_internal_sample[int(n_star*kom):]) )
        fLz_sample = (stellar_halo_k*np.tanh(Lz_sample/stellar_halo_chi)-stellar_halo_k+1)
        f_sample = (fQ_sample*kom + (1-kom))*fLz_sample

        # Save the values
        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            fE,
            fL,
            fLz,
            f,
            fE_sample,
            fL_sample,
            fLz_sample,
            f_sample
        )
        df_val_data_om2.append(_data)

# Save the data as a pickle
header = ['z0_sid','major_acc_sid','major_mlpid','merger_number',
          'fE','fL','fLz','f',
          'fE_sample','fL_sample','fLz_sample','f_sample']
with open(os.path.join(analysis_dir,'df_val_data_cb.pkl'),'wb') as handle:
    pickle.dump([header,df_val_data_cb], handle)
with open(os.path.join(analysis_dir,'df_val_data_om.pkl'),'wb') as handle:
    pickle.dump([header,df_val_data_om], handle)
with open(os.path.join(analysis_dir,'df_val_data_om2.pkl'),'wb') as handle:
    pickle.dump([header,df_val_data_om2], handle)

# Also save as a structured array
df_val_data_dtype = np.dtype([
    ('z0_sid',int),
    ('major_acc_sid',int),
    ('major_mlpid',int),
    ('merger_number',int),
    ('fE',object),
    ('fL',object),
    ('fLz',object),
    ('f',object),
    ('fE_sample',object),
    ('fL_sample',object),
    ('fLz_sample',object),
    ('f_sample',object)
    ])
df_val_data_cb = np.array(df_val_data_cb, dtype=df_val_data_dtype)
df_val_data_om = np.array(df_val_data_om, dtype=df_val_data_dtype)
df_val_data_om2 = np.array(df_val_data_om2, dtype=df_val_data_dtype)
np.save(os.path.join(analysis_dir,'df_val_data_cb.npy'), df_val_data_cb)
np.save(os.path.join(analysis_dir,'df_val_data_om.npy'), df_val_data_om)
np.save(os.path.join(analysis_dir,'df_val_data_om2.npy'), df_val_data_om2)

### Wrangle the data

In [ ]:
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Load the structured arrays
df_val_data_cb = np.load(os.path.join(analysis_dir,'df_val_data_cb.npy'),
    allow_pickle=True)
df_val_data_om = np.load(os.path.join(analysis_dir,'df_val_data_om.npy'),
    allow_pickle=True)
df_val_data_om2 = np.load(os.path.join(analysis_dir,'df_val_data_om2.npy'),
    allow_pickle=True)

# Load the merger information
merger_data = np.load(os.path.join(analysis_dir,'merger_data.npy'), 
    allow_pickle=True)

checks = True
if checks:
    assert np.all(df_val_data_cb['z0_sid'] ==df_val_data_om['z0_sid'] ), \
        'Something went wrong'
    assert np.all(df_val_data_cb['major_acc_sid'] ==df_val_data_om['major_acc_sid'] ), \
        'Something went wrong'
    assert np.all(df_val_data_cb['major_mlpid'] ==df_val_data_om['major_mlpid'] ), \
        'Something went wrong'
    assert np.all(df_val_data_cb['merger_number'] ==df_val_data_om2['merger_number'] ), \
        'Something went wrong'
    assert np.all(df_val_data_cb['z0_sid'] ==df_val_data_om2['z0_sid'] ), \
        'Something went wrong'
    assert np.all(df_val_data_cb['major_acc_sid'] ==df_val_data_om2['major_acc_sid'] ), \
        'Something went wrong'
    assert np.all(df_val_data_cb['z0_sid'] == merger_data['z0_sid']), \
        'Something went wrong'

In [ ]:
# Wrange the input loglikelihood a bit and construct a loglikelihood data
# structured array

loglike_data = []

for i in range(len(merger_data)):
    z0_sid = merger_data['z0_sid'][i]
    major_acc_sid = merger_data['major_acc_sid'][i]
    major_mlpid = merger_data['major_mlpid'][i]
    merger_number = merger_data['merger_number'][i]

    # Mask out negative values
    f_mask = (df_val_data_cb['f'][i] > 0) &\
                (df_val_data_om['f'][i] > 0) &\
                (df_val_data_om2['f'][i] > 0) &\
                (df_val_data_cb['f_sample'][i] > 0) &\
                (df_val_data_om['f_sample'][i] > 0) &\
                (df_val_data_om2['f_sample'][i] > 0)
    ll_cb = np.log(df_val_data_cb['f'][i][f_mask])
    ll_om = np.log(df_val_data_om['f'][i][f_mask])
    ll_om2 = np.log(df_val_data_om2['f'][i][f_mask])
    ll_cb_sample = np.log(df_val_data_cb['f_sample'][i][f_mask])
    ll_om_sample = np.log(df_val_data_om['f_sample'][i][f_mask])
    ll_om2_sample = np.log(df_val_data_om2['f_sample'][i][f_mask])
    ll_N = np.sum(f_mask)
    
    # Compute the differences
    lld_cb_om = np.sum(ll_cb) - np.sum(ll_om)
    lld_cb_om2 = np.sum(ll_cb) - np.sum(ll_om2)
    lld_om_om2 = np.sum(ll_om) - np.sum(ll_om2)
    lld_cb_cb_sample = np.sum(ll_cb) - np.sum(ll_cb_sample)
    lld_om_om_sample = np.sum(ll_om) - np.sum(ll_om_sample)
    lld_om2_om2_sample = np.sum(ll_om2) - np.sum(ll_om2_sample)
    lld_cb_om_self = np.sum(ll_cb) - np.sum(ll_om_sample)
    lld_cb_om_self = (np.sum(ll_cb) - np.sum(ll_cb_sample)) -\
                        (np.sum(ll_om) - np.sum(ll_om_sample))
    lld_cb_om2_self = (np.sum(ll_cb) - np.sum(ll_cb_sample)) -\
                        (np.sum(ll_om2) - np.sum(ll_om2_sample))
    lld_om_om2_self = (np.sum(ll_om) - np.sum(ll_om_sample)) -\
                        (np.sum(ll_om2) - np.sum(ll_om2_sample))    

    # Load into the structured array
    _data = (
        z0_sid,
        major_acc_sid,
        major_mlpid,
        merger_number,
        ll_cb,
        ll_om,
        ll_om2,
        ll_cb_sample,
        ll_om_sample,
        ll_om2_sample,
        ll_N,
        lld_cb_om,
        lld_cb_om2,
        lld_om_om2,
        lld_cb_cb_sample,
        lld_om_om_sample,
        lld_om2_om2_sample,
        lld_cb_om_self,
        lld_cb_om2_self,
        lld_om_om2_self
    )
    loglike_data.append(_data)

loglike_data_dtype = np.dtype([('z0_sid',int),
                               ('major_acc_sid',int),
                               ('major_mlpid',int),
                               ('merger_number',int),
                               ('ll_cb',object),
                               ('ll_om',object),
                               ('ll_om2',object),
                               ('ll_cb_sample',object),
                               ('ll_om_sample',object),
                               ('ll_om2_sample',object),
                               ('ll_N',int),
                               ('lld_cb_om',float),
                               ('lld_cb_om2',float),
                               ('lld_om_om2',float),
                               ('lld_cb_self',float),
                               ('lld_om_self',float),
                               ('lld_om2_self',float),
                               ('lld_cb_om_self',float),
                               ('lld_cb_om2_self',float),
                               ('lld_om_om2_self',float)
                               ])
loglike_data = np.array(loglike_data,dtype=loglike_data_dtype)

### Try and make some plots without relying on the 1/N factor

In [ ]:
# Will need merger data for this
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)
merger_data = np.load(os.path.join(analysis_dir,'merger_data.npy'), 
    allow_pickle=True)

# Validate it vs lldata
assert np.all( merger_data['major_mlpid'] == loglike_data['major_mlpid'] )

In [ ]:
# Some statistics about the range of values
llbins = [[-np.inf,-1000],
          [-1000,-100],
          [-100,0],
          [0,100],
          [100,1000],
          [1000,np.inf],
         ]

for i in range(len(llbins)):
    print(f'Number of values in bin ',llbins[i])
    print(f'CB-OM: {np.sum((loglike_data["lld_cb_om"] > llbins[i][0]) & (loglike_data["lld_cb_om"] < llbins[i][1]))}')
    print(f'CB-OM2: {np.sum((loglike_data["lld_cb_om2"] > llbins[i][0]) & (loglike_data["lld_cb_om2"] < llbins[i][1]))}')
    print(f'OM-OM2: {np.sum((loglike_data["lld_om_om2"] > llbins[i][0]) & (loglike_data["lld_om_om2"] < llbins[i][1]))}')
    print(f'CB-SIM: {np.sum((loglike_data["lld_cb_self"] > llbins[i][0]) & (loglike_data["lld_cb_self"] < llbins[i][1]))}')
    print(f'OM-SIM: {np.sum((loglike_data["lld_om_self"] > llbins[i][0]) & (loglike_data["lld_om_self"] < llbins[i][1]))}')
    print(f'OM2-SIM: {np.sum((loglike_data["lld_om2_self"] > llbins[i][0]) & (loglike_data["lld_om2_self"] < llbins[i][1]))}')
    print(f'CB-OM-SIM: {np.sum((loglike_data["lld_cb_om_self"] > llbins[i][0]) & (loglike_data["lld_cb_om_self"] < llbins[i][1]))}')
    

### Make a similar comparison with a symlog plot

In [ ]:
s = 10
facecolor = 'none'
edgecolor = 'Black'
ticklabelsize = 6
axislabelsize = 11
legendfontsize = 5
columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()

### CB vs OM
fig = plt.figure(figsize=(columnwidth, 2))
ax = fig.subplots(nrows=1, ncols=1)

lld_cb_om_mask = loglike_data['lld_cb_om'] > 0
ax.scatter(loglike_data['lld_cb_om'][lld_cb_om_mask], 
           merger_data['anisotropy'][lld_cb_om_mask], 
           facecolor='none', edgecolor=edgecolor, s=s, marker='^')
ax.scatter(loglike_data['lld_cb_om'][~lld_cb_om_mask], 
           merger_data['anisotropy'][~lld_cb_om_mask],
           facecolor=facecolor, edgecolor=edgecolor, s=s, marker='o')

ax.set_ylabel(r'$\beta$', fontsize=axislabelsize)
ax.set_ylim(-0.1, 1.0)

ax.tick_params(labelsize=ticklabelsize)
ax.set_xlabel(r'$\log \mathcal{L}_\mathrm{CM} - \log \mathcal{L}_\mathrm{OM}$', 
    fontsize=axislabelsize)
ax.set_xscale('symlog', linthresh=1)
ax.set_xlim(-1e6,1e6)
ax.set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])

fig.tight_layout()
# fig.subplots_adjust(hspace=0.1)
# fig.savefig('./fig/loglike_number_beta_comparison.pdf')
fig.show()


### OM vs OM2
fig = plt.figure(figsize=(columnwidth, 2))
ax = fig.subplots(nrows=1, ncols=1)

ax.scatter(loglike_data['lld_om_om2'][lld_cb_om_mask], 
           merger_data['anisotropy'][lld_cb_om_mask], 
           facecolor='none', edgecolor=edgecolor, s=s, marker='^')
ax.scatter(-loglike_data['lld_om_om2'][~lld_cb_om_mask], 
           merger_data['anisotropy'][~lld_cb_om_mask],
           facecolor=facecolor, edgecolor=edgecolor, s=s, marker='o')

ax.set_ylabel(r'$\beta$', fontsize=axislabelsize)
ax.set_ylim(-0.1, 1.0)

ax.tick_params(labelsize=ticklabelsize)
ax.set_xlabel(r'$\log \mathcal{L}_\mathrm{OM2} - \log \mathcal{L}_\mathrm{OM}$', 
    fontsize=axislabelsize)
ax.set_xscale('symlog', linthresh=1)
ax.set_xlim(-1e6,1e6)
ax.set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])

fig.tight_layout()
# fig.subplots_adjust(hspace=0.1)
# fig.savefig('./fig/loglike_number_beta_comparison.pdf')
fig.show()

In [ ]:
s = 10
facecolor = 'none'
edgecolor = 'Black'
ticklabelsize = 6
axislabelsize = 11
legendfontsize = 4
hist_colors = ['DodgerBlue','Red']
hist_linestyles = ['solid','dashed']
hist_linewidths = [2,1]

columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()
fig = plt.figure(figsize=(columnwidth, 3))
gs = fig.add_gridspec(2,7)
axs = [fig.add_subplot(gs[0,0:5]),
       fig.add_subplot(gs[1,0:5])]
raxs = [fig.add_subplot(gs[0,5:]),
        fig.add_subplot(gs[1,5:])]

# ax = fig.subplots(nrows=1, ncols=1)

## Plot the main panels: anisotropy and logarithmic stellar mass
axs[0].scatter(loglike_data['lld_cb_om'], merger_data['anisotropy'], 
           facecolor='none', edgecolor=edgecolor, s=s)
axs[0].set_ylabel(r'$\beta$', fontsize=axislabelsize)
axs[0].set_ylim(-0.1, 1.0)
axs[0].tick_params(labelsize=ticklabelsize, labelbottom=False)
# axs[0].set_xlabel(r'$\Delta \log \mathcal{L}$', fontsize=axislabelsize)
# axs[0].set_xscale('symlog', linthresh=1)
# axs[0].set_xlim(-1e7,1e7)
axs[0].set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])

axs[1].scatter(loglike_data['lld_cb_om'], np.log10(merger_data['star_mass']), 
           facecolor='none', edgecolor=edgecolor, s=s)
axs[1].set_ylabel(r'$\log_{10}(\mathrm{M}_{\star} / \mathrm{M}_{\odot})$', 
    fontsize=axislabelsize)
axs[1].set_ylim(6.5, 11)
axs[1].tick_params(labelsize=ticklabelsize)
# axs[1].set_xlabel(r'$\Delta \log \mathcal{L}$', fontsize=axislabelsize)
axs[1].set_xlabel(r'$\mathcal{L}_\mathrm{CB} - \mathcal{L}_\mathrm{OM}$', 
    fontsize=axislabelsize)
# axs[1].set_xscale('symlog', linthresh=1)
# axs[1].set_xlim(-1e7,1e7)
# axs[1].set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])

for ax in axs:
    ax.set_xscale('symlog', linthresh=1)
    ax.set_xlim(-1e7,1e7)
    ax.set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])
    ax.axvline(0, color='Gray', ls='dashed', lw=1, zorder=1)
axs[1].set_xticklabels([r'$-10^{6}$',r'$-10^{4}$',r'$-10^{2}$',r'$-1$',r'$0$',
                        r'$1$',r'$10^{2}$',r'$10^{4}$',r'$10^{6}$'])

## Plot the margins on the side panels, separated by positive/negative loglike
mask = loglike_data['lld_cb_om'] > 0
raxs[0].hist(merger_data['anisotropy'][mask], range=[-0.1,1], bins=11, 
             histtype='step', color=hist_colors[0], lw=hist_linewidths[0], 
             ls=hist_linestyles[0], density=True, orientation='horizontal')
raxs[0].hist(merger_data['anisotropy'][~mask], range=[-0.1,1], bins=11, 
             histtype='step', color=hist_colors[1], lw=hist_linewidths[1], 
             ls=hist_linestyles[1], density=True, orientation='horizontal')
raxs[0].set_ylim(-0.1, 1.0)
raxs[0].tick_params(labelleft=False, labelbottom=True, 
    labeltop=False, labelsize=ticklabelsize)
# raxs[0].set_xlabel(r'p($\beta$)', fontsize=axislabelsize)
# raxs[0].xaxis.set_label_position('top')
# raxs[0].set_xlim(0, 20)

raxs[1].hist(np.log10(merger_data['star_mass'][mask]), range=[6.5,10.5], bins=10,
                histtype='step', color=hist_colors[0], lw=hist_linewidths[0], 
                ls=hist_linestyles[0], density=True, orientation='horizontal')
raxs[1].hist(np.log10(merger_data['star_mass'][~mask]), range=[6.5,10.5], bins=10,
                histtype='step', color=hist_colors[1], lw=hist_linewidths[1], 
                ls=hist_linestyles[1], density=True, orientation='horizontal')
raxs[1].set_ylim(6.5, 11)
raxs[1].tick_params(labelleft=False, labelsize=ticklabelsize)
raxs[1].set_xlabel(r'Density', fontsize=axislabelsize)
# raxs[1].set_xlim(0, 20)

# Annotate the top plot to show the number of points in either group
axs[0].annotate(int(np.sum(mask)), xy=(0.56,0.9), xycoords='axes fraction', 
                ha='center', va='center', fontsize=ticklabelsize, 
                color=hist_colors[0])
axs[0].annotate(int(np.sum(~mask)), xy=(0.46,0.9), xycoords='axes fraction', 
                ha='center', va='center', fontsize=ticklabelsize,
                color=hist_colors[1])

# Legend
raxs[0].plot([], [], color=hist_colors[0], linewidth=hist_linewidths[0],
    linestyle=hist_linestyles[0], label=r'$\Delta \mathrm{log} \mathcal{L} > 0$')
raxs[0].plot([], [], color=hist_colors[1], linewidth=hist_linewidths[1],
    linestyle=hist_linestyles[1], label=r'$\Delta \mathrm{log} \mathcal{L} < 0$')
raxs[0].legend(loc='upper right', fontsize=legendfontsize, handlelength=2, 
    frameon=False)

fig.tight_layout()
fig.subplots_adjust(hspace=0.15, wspace=0.15)
fig.savefig('./fig/loglike_symlog_starmass_beta_comparison_cb_om.pdf')
fig.show()

In [ ]:
s = 10
facecolor = 'none'
edgecolor = 'Black'
ticklabelsize = 6
axislabelsize = 11
legendfontsize = 4
hist_colors = ['DodgerBlue','Red']
hist_linestyles = ['solid','dashed']
hist_linewidths = [2,1]

columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()
fig = plt.figure(figsize=(columnwidth, 3))
gs = fig.add_gridspec(2,7)
axs = [fig.add_subplot(gs[0,0:5]),
       fig.add_subplot(gs[1,0:5])]
raxs = [fig.add_subplot(gs[0,5:]),
        fig.add_subplot(gs[1,5:])]

# ax = fig.subplots(nrows=1, ncols=1)

## Plot the main panels: anisotropy and logarithmic stellar mass
axs[0].scatter(-loglike_data['lld_om_om2'], merger_data['anisotropy'], 
           facecolor='none', edgecolor=edgecolor, s=s)
axs[0].set_ylabel(r'$\beta$', fontsize=axislabelsize)
axs[0].set_ylim(-0.1, 1.0)
axs[0].tick_params(labelsize=ticklabelsize, labelbottom=False)
# axs[0].set_xlabel(r'$\Delta \log \mathcal{L}$', fontsize=axislabelsize)
# axs[0].set_xscale('symlog', linthresh=1)
# axs[0].set_xlim(-1e7,1e7)
axs[0].set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])

axs[1].scatter(-loglike_data['lld_om_om2'], np.log10(merger_data['star_mass']), 
           facecolor='none', edgecolor=edgecolor, s=s)
axs[1].set_ylabel(r'$\log_{10}(\mathrm{M}_{\star} / \mathrm{M}_{\odot})$', 
    fontsize=axislabelsize)
axs[1].set_ylim(6.5, 11)
axs[1].tick_params(labelsize=ticklabelsize)
# axs[1].set_xlabel(r'$\Delta \log \mathcal{L}$', fontsize=axislabelsize)
axs[1].set_xlabel(r'$\mathcal{L}_\mathrm{OM2} - \mathcal{L}_\mathrm{OM}$', 
    fontsize=axislabelsize)
# axs[1].set_xscale('symlog', linthresh=1)
# axs[1].set_xlim(-1e7,1e7)
# axs[1].set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])

for ax in axs:
    ax.set_xscale('symlog', linthresh=1)
    ax.set_xlim(-1e7,1e7)
    ax.set_xticks([-1e6,-1e4,-1e2,-1e0,0,1e0,1e2,1e4,1e6])
    ax.axvline(0, color='Gray', ls='dashed', lw=1, zorder=1)
axs[1].set_xticklabels([r'$-10^{6}$',r'$-10^{4}$',r'$-10^{2}$',r'$-1$',r'$0$',
                        r'$1$',r'$10^{2}$',r'$10^{4}$',r'$10^{6}$'])

## Plot the margins on the side panels, separated by positive/negative loglike
mask = -loglike_data['lld_om_om2'] > 0
raxs[0].hist(merger_data['anisotropy'][mask], range=[-0.1,1], bins=11, 
             histtype='step', color=hist_colors[0], lw=hist_linewidths[0], 
             ls=hist_linestyles[0], density=True, orientation='horizontal')
raxs[0].hist(merger_data['anisotropy'][~mask], range=[-0.1,1], bins=11, 
             histtype='step', color=hist_colors[1], lw=hist_linewidths[1], 
             ls=hist_linestyles[1], density=True, orientation='horizontal')
# raxs[0].hist(merger_data['anisotropy'][mask], range=[-0.1,1], bins=11, 
#              histtype='step', color=hist_colors[0], lw=1, density=True,
#              orientation='horizontal')
# raxs[0].hist(merger_data['anisotropy'][~mask], range=[-0.1,1], bins=11, 
#              histtype='step', color=hist_colors[1], lw=1, density=True,
#              orientation='horizontal')
raxs[0].set_ylim(-0.1, 1.0)
raxs[0].tick_params(labelleft=False, labelbottom=True, 
    labeltop=False, labelsize=ticklabelsize)
# raxs[0].set_xlabel(r'p($\beta$)', fontsize=axislabelsize)
# raxs[0].xaxis.set_label_position('top')
# raxs[0].set_xlim(0, 20)

raxs[1].hist(np.log10(merger_data['star_mass'][mask]), range=[6.5,10.5], bins=10,
                histtype='step', color=hist_colors[0], lw=hist_linewidths[0], 
                ls=hist_linestyles[0], density=True, orientation='horizontal')
raxs[1].hist(np.log10(merger_data['star_mass'][~mask]), range=[6.5,10.5], bins=10,
                histtype='step', color=hist_colors[1], lw=hist_linewidths[1], 
                ls=hist_linestyles[1], density=True, orientation='horizontal')
# raxs[1].hist(np.log10(merger_data['star_mass'][mask]), range=[6.5,10.5], bins=10,
#                 histtype='step', color=hist_colors[0], lw=1, density=True,
#                 orientation='horizontal')
# raxs[1].hist(np.log10(merger_data['star_mass'][~mask]), range=[6.5,10.5], bins=10,
#                 histtype='step', color=hist_colors[1], lw=1, density=True,
#                 orientation='horizontal')
raxs[1].set_ylim(6.5, 11)
raxs[1].tick_params(labelleft=False, labelsize=ticklabelsize)
raxs[1].set_xlabel(r'Density', fontsize=axislabelsize)
# raxs[1].set_xlim(0, 20)

# Annotate the top plot to show the number of points in either group
axs[0].annotate(int(np.sum(mask)), xy=(0.56,0.9), xycoords='axes fraction', 
                ha='center', va='center', fontsize=ticklabelsize, 
                color=hist_colors[0])
axs[0].annotate(int(np.sum(~mask)), xy=(0.46,0.9), xycoords='axes fraction', 
                ha='center', va='center', fontsize=ticklabelsize,
                color=hist_colors[1])

# Legend
# raxs[0].plot([], [], color=hist_colors[0], linewidth=hist_linewidths[0],
#     linestyle=hist_linestyles[0], label=r'$\Delta \mathrm{log} \mathcal{L} > 0$')
# raxs[0].plot([], [], color=hist_colors[1], linewidth=hist_linewidths[1],
#     linestyle=hist_linestyles[1], label=r'$\Delta \mathrm{log} \mathcal{L} < 0$')
# raxs[0].legend(loc='upper right', fontsize=legendfontsize, handlelength=2, 
#     frameon=False)
# raxs[0].plot([], [], color=hist_colors[0], label=r'$\Delta \mathrm{log} \mathcal{L} > 0$')
# raxs[0].plot([], [], color=hist_colors[1], label=r'$\Delta \mathrm{log} \mathcal{L} < 0$')
# raxs[0].legend(loc='upper right', fontsize=legendfontsize, handlelength=0.5, 
#     frameon=False)

fig.tight_layout()
fig.subplots_adjust(hspace=0.15, wspace=0.15)
fig.savefig('./fig/loglike_symlog_starmass_beta_comparison_om2_om.pdf')
fig.show()

In [ ]:
# def make_loglike_diff_violin_plot(merger_data_key, is_log=False, ylabel=''):
#     # A single plot that can be used to query any quantity in merger_data

#     llbins = [[-np.inf,-1000],
#             [-1000,-100],
#             [-100,0],
#             [0,100],
#             [100,1000],
#             [1000,np.inf],
#             ]
#     llkeys = ['lld_cb_om','lld_cb_self','lld_om_self','lld_cb_om_self']
#     labels = [r'${\rm CB-OM}$', r'${\rm CB-SIM}$', r'${\rm OM-SIM}$', r'${\rm (CB-SIM)-(OM-SIM)}$']
#     # labels = [r'$\mathcal{L}_{\rm CB-OM}$',
#     #           r'$\mathcal{L}_{\rm CB-S}$',
#     #           r'$\mathcal{L}_{\rm OM-S}$',
#     #           r'$\mathcal{L}_{\rm (OM-S)-(CB-S)}$'
#     #           ]
#     # labels but in scientific notation
#     x_labels = [r'$< -10^3$', r'$-10^3$ to $-10^2$', r'$-10^2$ to $0$',
#             r'$0$ to $10^2$', r'$10^2$ to $10^3$', r'$> 10^3$']
#     # labels = [r'$< -1000$', r'$-1000$ to $-100$', r'$-100$ to $0$',
#     #           r'$0$ to $100$', r'$100$ to $1000$', r'$> 1000$']
#     s = 10
#     colors = ['Red', 'DarkOrange', 'DodgerBlue', 'Purple']
#     edgecolor = 'Black'
#     offset = [-0.24,-0.08,0.08,0.24]
#     ticklabelsize = 6
#     axislabelsize = 12
#     legendfontsize = 6

#     columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()
#     fig = plt.figure(figsize=(columnwidth, 3))
#     axs = fig.subplots(nrows=2, ncols=1)

#     for i in range(len(llbins)):
#         for j in range(len(llkeys)):
#             x = lldata[llkeys[j]]
#             mask = (x > llbins[i][0]) & (x < llbins[i][1])
#             x = x[mask]
#             N = len(x)
#             if i == 0:
#                 axs[0].scatter(i+offset[j], N, s=20, color=colors[j], 
#                     label=labels[j])
#             else:
#                 axs[0].scatter(i+offset[j], N, s=20, color=colors[j])
#                 # facecolor=colors[j], edgecolor=edgecolor)

#     axs[0].set_xticks(range(len(llbins)))
#     axs[0].tick_params(labelsize=ticklabelsize, labelbottom=False)
#     axs[0].set_ylabel('N', fontsize=axislabelsize)
#     axs[0].set_ylim(0, 60)
#     axs[0].legend(loc='upper right', fontsize=legendfontsize)

#     for i in range(len(llbins)):
#         for j in range(len(llkeys)):
#             x = lldata[llkeys[j]]
#             mask = (x > llbins[i][0]) & (x < llbins[i][1])
#             y = merger_data[merger_data_key][mask]
#             if is_log:
#                 y = np.log10(y)
#             ly, my, uy = np.percentile(y, [16,50,84])
#             axs[1].errorbar(i+offset[j], my, yerr=[[my-ly],[uy-my]], 
#                 fmt='o', color=colors[j], ms=5, lw=1, capsize=0, capthick=0)
#             # axs[1].scatter(i+offset[j], N, s=s, facecolor=colors[j], 
#             #     edgecolor=edgecolor)

#     for j in range(len(llkeys)):
#         axs[1].scatter([], [], s=s, facecolor=colors[j], edgecolor=edgecolor, 
#             label=labels[j])

#     axs[1].set_xticks(range(len(llbins)))
#     axs[1].set_xticklabels(x_labels)
#     axs[1].tick_params(labelsize=ticklabelsize, rotation=30)
#     axs[1].set_xlabel(r'$\Delta \log \mathcal{L}$', fontsize=axislabelsize)
#     axs[1].set_ylabel(ylabel, fontsize=axislabelsize)
#     # axs[1].set_ylim(0,0.8)

#     return fig,axs
# # fig.show()

In [ ]:
# fig, axs = make_loglike_diff_violin_plot('star_mass_ratio', is_log=True, ylabel=r'star $\log M_{s}/M_{p}$')
# fig.show()

In [ ]:
# fig, axs = make_loglike_diff_violin_plot('merger_redshift', ylabel=r'$z$')
# fig.show()